In [ ]:
from google.cloud import bigquery
import os
import pandas as pd



# Replace with the path to your JSON key file
key_path = "./dashboard-254616-5def36d755e0.json"

# Set environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path

# Initialize BigQuery client
client = bigquery.Client()

In [3]:
query = """
  SELECT * FROM `dashboard-254616.donations.donations_time_period_with_prov`
"""
query_job = client.query(query)

results = query_job.result()

df = results.to_dataframe()  
df.head()

,Donation_ID,Method,Deposit_Date,Contact_ID,Campaign_ID,Revenue_Account,Amount,Recurring_Donation,month,year,...,total_amount,donor_level,channel,longest_consecutive_years,campaign_name,gift_number,FYTD,Fiscal_Year,TimePeriod,Prov
0,1000143,PayPal,2020-01-29 00:00:00+00:00,1000142.0,2091.0,Website,25.0,No,1,2020,...,440.0,broadbase,Website,4,FED-EN-general-DONATE+main donation page,1,False,2020,Calendar YTD,SK
1,1113887,MasterCard,2020-08-28 00:00:00+00:00,1000142.0,NaN,Website,10.0,No,8,2020,...,440.0,broadbase,Website,4,None,2,True,2020,Calendar YTD,SK
2,1120105,MasterCard,2020-09-29 00:00:00+00:00,1000142.0,NaN,Website,10.0,Yes,9,2020,...,440.0,broadbase,Website,4,None,3,True,2020,Calendar YTD,SK
3,1147685,MasterCard,2020-10-29 00:00:00+00:00,1000142.0,NaN,Website,10.0,Yes,10,2020,...,440.0,broadbase,Website,4,None,4,True,2020,Calendar YTD,SK
4,1155514,MasterCard,2020-11-27 00:00:00+00:00,1000142.0,NaN,Website,10.0,Yes,11,2020,...,440.0,broadbase,Website,4,None,5,True,2020,Calendar YTD,SK


In [7]:
df['Fiscal_Year'].unique()

array(['2020', '2021', '2022', '2023', '2024', '2025', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2026',
       'Nov/23-Nov/24', 'Nov/22-Nov/23', 'Nov/24-Nov/25', 'Nov/21-Nov/22',
       'Nov/19-Nov/20', 'Nov/20-Nov/21', 'Nov/17-Nov/18', 'Nov/13-Nov/14',
       'Nov/14-Nov/15', 'Nov/16-Nov/17', 'Nov/15-Nov/16', 'Nov/18-Nov/19'],
      dtype=object)

In [8]:
import plotly.express as px

# Remove records where Fiscal_Year contains 'Nov' (case-insensitive)
df_clean = df[~df['Fiscal_Year'].astype(str).str.contains('nov', case=False)]

revenue_per_year_account = df_clean.groupby(['Fiscal_Year','Revenue_Account'], as_index=False)['Amount'].sum()

revenue_per_year_account

revenue_per_year_account_chart = px.bar(revenue_per_year_account, x='Fiscal_Year', y='Amount', color='Revenue_Account')
revenue_per_year_account_chart.show()

In [9]:

revenue_per_year = df_clean.groupby(['Fiscal_Year'], as_index=False)['Amount'].sum()

revenue_per_year

revenue_per_year_chart = px.bar(revenue_per_year, x='Fiscal_Year', y='Amount')
revenue_per_year_chart.show()

In [10]:
unique_donors_per_year = df_clean.groupby(['Fiscal_Year'], as_index=False)['Contact_ID'].nunique()
unique_donors_per_year

unique_donors_per_year_chart = px.bar(unique_donors_per_year, x='Fiscal_Year', y='Contact_ID')
unique_donors_per_year_chart.show()



In [11]:
unique_donations_per_year = df_clean.groupby(['Fiscal_Year'], as_index=False)['Donation_ID'].nunique()
unique_donations_per_year

unique_donations_per_year_chart = px.bar(unique_donations_per_year, x='Fiscal_Year', y='Donation_ID')
unique_donations_per_year_chart.show()



In [12]:
average_donation_per_year = df_clean.groupby(['Fiscal_Year'], as_index=False)['Amount'].mean()
average_donation_per_year

average_donation_per_year_chart = px.bar(average_donation_per_year, x='Fiscal_Year', y='Amount')
average_donation_per_year_chart.show()



In [21]:
donations_per_donor_per_year = df_clean.groupby(['Fiscal_Year', 'Contact_ID'], as_index=False)['Donation_ID'].count().rename(columns={'Donation_ID': 'Donations_Count'})
avg_donations_per_donor_per_year = donations_per_donor_per_year.groupby(['Fiscal_Year'], as_index=False)['Donations_Count'].mean()
avg_donations_per_donor_per_year

avg_donations_per_donor_per_year_chart = px.bar(avg_donations_per_donor_per_year, x='Fiscal_Year', y='Donations_Count')
avg_donations_per_donor_per_year_chart.show()


